<a href="https://colab.research.google.com/github/ddekun/Intro_Neural_Networks/blob/lesson8/lesson8/hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в нейронные сети

### Урок 8. GAN

1. Обучите нейронную сеть любой архитектуры, которой не было на курсе, либо нейронную сеть разобранной архитектуры, но на том датасете, которого не было на уроках. Сделайте анализ того, что вам помогло в улучшения работы нейронной сети

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

**Строим слой семплирования sampling layer**

In [23]:
# стороим кастомный слой
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a unit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs  # с двумя параметрами на входе
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1] # определяем размер нашего пространства
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim)) # строим шум
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon # восстанавливаем пакет единиц информации, опираясь на параметры нашего распр

**Строим энкодер encoder**

In [24]:
latent_dim = 2 # берем пространство равное 2 для быстроты

encoder_inputs = keras.Input(shape=(28, 28, 1))
# строим сверточную модель
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
# строим выходы
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
# получилась модель: на входе картинка, на выходе 3 тензора
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_10 (Conv2D)             (None, 14, 14, 32)   320         ['input_2[0][0]']                
                                                                                                  
 conv2d_11 (Conv2D)             (None, 7, 7, 64)     18496       ['conv2d_10[0][0]']              
                                                                                                  
 flatten_1 (Flatten)            (None, 3136)         0           ['conv2d_11[0][0]']              
                                                                                            

**Строим свой decoder**

In [25]:
latent_inputs = keras.Input(shape=(latent_dim,))
# размерность 7 * 7 * 64 позволяет потом построить выход 28
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
# превращаем в тензор более сложной формы 4-мерный
x = layers.Reshape((7, 7, 64))(x)
# как тольо сказали strides=2, то увеличили ращмерность в 2 раза
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x) # 14x14
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x) # 28x28
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x) # одноканальная картинка 28x28
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_3 (Dense)             (None, 3136)              9408      
                                                                 
 reshape_1 (Reshape)         (None, 7, 7, 64)          0         
                                                                 
 conv2d_transpose_8 (Conv2DT  (None, 14, 14, 64)       36928     
 ranspose)                                                       
                                                                 
 conv2d_transpose_9 (Conv2DT  (None, 28, 28, 32)       18464     
 ranspose)                                                       
                                                                 
 conv2d_transpose_10 (Conv2D  (None, 28, 28, 1)        289 

**Создаем класс модель по Model с особым шагом обучения train_step**

In [26]:
# создаем модель как наследник класса Model
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data): # меняем на свое обучение функцию train_step
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape: # создаем объект для дифферецнирования
            z_mean, z_log_var, z = encoder(data)  # картинки пропускаем через энкодер
            reconstruction = decoder(z) # пропускаем через декодер
            # строим первый loss
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 28 * 28
            # строим второй loss
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5 # коэффициент обеспечивает равный вклад двух лоссов
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights) # говорим, что у модели будут тренироваться веса
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

**Учим VAE**

In [ ]:
(x_train, _), (x_test, _) = keras.datasets.fashion_mnist.load_data() # подгружаем FASHION-MNIST
mnist_units = np.concatenate([x_train, x_test], axis=0)
mnist_units = np.expand_dims(mnist_units, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam()) # задаем оптимайзер
vae.fit(mnist_units, epochs=30, batch_size=128) # проводим обучение

4422102/4422102 [==============================] - 1s 0us/step
Epoch 1/30
547/547 [==============================] - 150s 271ms/step - loss: 317.6309 - reconstruction_loss: 312.1427 - kl_loss: 5.4882
Epoch 2/30
547/547 [==============================] - 141s 258ms/step - loss: 277.2750 - reconstruction_loss: 272.9078 - kl_loss: 4.3672
Epoch 3/30
547/547 [==============================] - 148s 271ms/step - loss: 272.2064 - reconstruction_loss: 268.2615 - kl_loss: 3.9449
Epoch 4/30
163/547 [=======>......................] - ETA: 1:35 - loss: 270.3435 - reconstruction_loss: 266.4824 - kl_loss: 3.8610

**Результат предсказания для датасета одежды**

In [ ]:
import matplotlib.pyplot as plt

# на этапе предсказания энкодер не нужен
# По одной точке из скрытого пространства восстанавливаем объект, подавая через декодер
def plot_latent(encoder, decoder):
    # display a n*n 2D manifold of units
    n = 30
    unit_size = 28
    scale = 2.0 # диапазон пространства от минус 2 до 2
    figsize = 15
    figure = np.zeros((unit_size * n, unit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of unit classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            unit = x_decoded[0].reshape(unit_size, unit_size)
            figure[
                i * unit_size : (i + 1) * unit_size,
                j * unit_size : (j + 1) * unit_size,
            ] = unit

    plt.figure(figsize=(figsize, figsize))
    start_range = unit_size // 2
    end_range = n * unit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, unit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent(encoder, decoder)

**Посмотрим на кластеры в скрытом пространстве для одежды**

In [ ]:
def plot_label_clusters(encoder, data, labels):
    # display a 2D plot of the unit classes in the latent space
    z_mean, _, _ = encoder.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()


(x_train, y_train), _ = keras.datasets.fashion_mnist.load_data()
x_train = np.expand_dims(x_train, -1).astype("float32") / 255

plot_label_clusters(encoder, x_train, y_train)

Построим немного примеров генерации одежды:

для этого не нужен энкодер работа этой модели требует:
- выбрать случайные значения для точки скрытого пространства (2 координаты (среднее и логарифм от дисперсиии) - для построения в n (в нашем случае 28х28) координаты нового образца )
- подать их в декодер
- визуализировать результат

In [ ]:
def plot_unit_from_decoder(decoder, z, unit_size):
    # display a 2D plot of the unit classes in the latent space

    data = np.array([[z[0], z[1]]])
    unit = decoder.predict(data)
    unit = unit.reshape(unit_size, unit_size)

    plt.figure(figsize=(10, 10))
    plt.imshow(unit)
    plt.colorbar()
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()


plot_unit_from_decoder(decoder, [1.4,-1.3], 28)

**Увеличим размер скрытого пространства**

In [ ]:
latent_dim = 4

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(128, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 128, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 128))(x)
x = layers.Conv2DTranspose(128, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam()) # задаем оптимайзер
vae.fit(mnist_units, epochs=30, batch_size=128) # проводим обучение

По сравнению с сетью с 2 скрытыми слоями - сети с 6 скрытыми слоями удалось добиться уменьшения обеих составляющих ошибки. Лоссы стабилизировались.

**Визуализируем полученные результаты, зафиксировав несколько измерений.**

In [ ]:
def plot_latent_4(encoder, decoder):
    n = 30
    size = 28
    scale = 2.0
    figsize = 15
    figure = np.zeros((size * n, size * n))
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[1, 0, xi, yi]])
            x_decoded = decoder.predict(z_sample)
            pic = x_decoded[0].reshape(size, size)
            figure[
                i * size : (i + 1) * size,
                j * size : (j + 1) * size,
            ] = pic

    plt.figure(figsize=(figsize, figsize))
    start_range = size // 2
    end_range = (n - 1) * size + start_range + 1
    pixel_range = np.arange(start_range, end_range, size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[3]")
    plt.ylabel("z[4]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent_4(encoder, decoder)

In [ ]:
x_decoded = decoder.predict([[1, 0, -1.7, 1.7]])
plt.imshow(x_decoded[0,:,:,0])
plt.show()

**2. Сделайте краткий обзор научной работы, посвящённой алгоритму нейронных сетей, не рассматриваемому ранее на курсе. Проведите анализ: чем отличается выбранная архитектура от других? В чём плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при её применении на практике?**

**Deep Q Network (DQN)**

Об успехах Google Deepmind сейчас знают и говорят. Алгоритмы DQN (Deep Q-Network) побеждают Человека с неплохим отрывом всё в большее количество игр. Достижения последних лет впечатляют: буквально за десятки минут обучения, алгоритмы учатся выигрывать человека в понг и другие игры Atari. Недавно вышли в третье измерение — побеждают человека в DOOM в реальном времени, а также учатся управлять машинами и вертолетами.

Алгоритм DQN (Deep Q-Network) algorithm впервые появился в 2015г. Он позволил добиться столь впечатляющих успехов путем сочетания Reinforcement learning (RL) с нейронными сетями глубокого обучения.

DQN и Qleanring - аналогичные алгоритмы, основанные на итерации значения, но в обычном Q-обучении, когда состояние и пространство действия дискретны, а размерность невелика, Q-таблица может использоваться для хранения значения Q каждой пары действий состояния, и когда состояние и пространство действия непрерывны в высоком измерении, очень трудно использовать Q-таблицу без слишком большого пространства действия и состояния. Однако, можно перевести обновление Q-таблицы в проблему подбора функций, подгоняя функцию вместо Q-таблицы для генерации значений Q. В результате чего, похожие состояния будут вызывать похожие выходные действия.

Таким образом, глубокая нейронная сеть хорошо влияет на извлечение сложных функций. Такое влияние даёт возможность комбинировать DeepLearning (DL) и Reinforcement Learning (RL).

**Комбинация DL и RL имеет следующие проблемы:**

- Шум и задержка, в связи с этим во многих состояниях значения вознаграждения состояния равны 0, что является редкими выборками.
- Каждый образец DL не зависит друг от друга, и значение текущего состояния RL зависит от возвращаемого значения последнего состояния.
- При использовании нелинейной сети для представления функции значения, может возникнуть нестабильность

Два основных инструмента в DQN решают вышеуказанные проблемы

- Использование награды для создания тегов через Q-Learning
- Воспроизведение опыта (пул опыта) для решения проблемы корреляции и нестатического распределения.

Пул опыта в DQN используется для изучения предыдущего опыта, и поскольку Q обучение это метод автономного обучения то такой метод позволяет не только учиться на прошлом опыте но так же учиться без использования прошлого опыта, что уже почти фантастика.

**Основные преимущества Deep Q Network**

- Случайное добавление предыдущего опыта в учебный процесс сделает нейронную сеть более эффективной.
- Пул опыта решает проблему релевантности и нестатического распределения.

Основной недостаток

С ростом числа циклов обучения происходит резкое увеличение потребляемых ресурсов при этом ошибка уменьшается незначительно, а в некоторых случаях может и возрастать.

В этом недостатке и заключается основная трудность в применении алгоритма на практике.